In [ ]:
#requried libraries
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Embedding,LSTM,Dense
from tensorflow.keras.models import load_model



In [3]:
data=pd.read_csv('medical_data.csv')
data.head()

,Patient_Problem,Disease,Prescription
0,"Constant fatigue and muscle weakness, struggli...",Chronic Fatigue Syndrome,"Cognitive behavioral therapy, graded exercise ..."
1,"Frequent severe migraines, sensitivity to ligh...",Migraine with Aura,"Prescription triptans, avoid triggers like bri..."
2,"Sudden weight gain and feeling cold, especiall...",Hypothyroidism,Levothyroxine to regulate thyroid hormone levels.
3,"High fever, sore throat, and swollen lymph nod...",Mononucleosis,"Rest and hydration, ibuprofen for pain."
4,"Excessive thirst and frequent urination, dry m...",Diabetes Mellitus,Insulin therapy and lifestyle changes.


In [4]:
tokenizer=Tokenizer(num_words=5000,oov_token="<OOV>")
tokenizer.fit_on_texts(data['Patient_Problem'])
sequences=tokenizer.texts_to_sequences(data['Patient_Problem'])

In [5]:
max_length=max(len(x) for x in sequences)
padded_seq=pad_sequences(sequences,maxlen=max_length,padding='post')

In [6]:
#labelEnconder
label_encode_disease=LabelEncoder()
label_encode_prescription=LabelEncoder()

disease_labels=label_encode_disease.fit_transform(data['Disease'])
prescription_labels=label_encode_prescription.fit_transform(data['Prescription'])

#converting to categorical
disease_labels_cat=to_categorical(disease_labels)
prescription_labels_cat=to_categorical(prescription_labels)

In [7]:
#combining labels into a multi_label target varibles 
Y=np.hstack((disease_labels_cat,prescription_labels_cat))

*Model Building using LSTM*

In [9]:
input_layer=Input(shape=(max_length,))

embedding=Embedding(input_dim=5000,output_dim=64)(input_layer)
lstm_layer=LSTM(64)(embedding)
disease_output=Dense(len(label_encode_disease.classes_),activation='softmax',name='disease_output')(lstm_layer)

prescription_output=Dense(len(label_encode_prescription.classes_),activation='softmax',name='prescription_output')(lstm_layer)

In [12]:
#compling the model

model=Model(inputs=input_layer,outputs=[disease_output,prescription_output])

model.compile(
    loss={'disease_output': 'categorical_crossentropy', 
    'prescription_output': 'categorical_crossentropy'},
    optimizer='adam',
    metrics={'disease_output': ['accuracy'], 'prescription_output': ['accuracy']}
)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 17)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 17, 64)    │    320,000 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 64)        │     33,024 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ disease_output      │ (None, 178)       │     11,570 │ lstm[0][0]        │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prescription_output │ (None, 388)       │     25,220 │ lstm[0][0]        │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 389,814 (1.49 MB)

 Trainable params: 389,814 (1.49 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
#training the model
model.fit(padded_seq,{'disease_output':disease_labels_cat, 'prescription_output':prescription_labels_cat},epochs=150,batch_size=32)

Epoch 1/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - disease_output_accuracy: 0.0295 - disease_output_loss: 5.1783 - loss: 11.1455 - prescription_output_accuracy: 0.0000e+00 - prescription_output_loss: 5.9675
Epoch 2/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - disease_output_accuracy: 0.0344 - disease_output_loss: 5.1595 - loss: 11.1231 - prescription_output_accuracy: 0.0000e+00 - prescription_output_loss: 5.9635
Epoch 3/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - disease_output_accuracy: 0.0344 - disease_output_loss: 5.1040 - loss: 11.0752 - prescription_output_accuracy: 0.0025 - prescription_output_loss: 5.9728    
Epoch 4/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - disease_output_accuracy: 0.0344 - disease_output_loss: 4.9756 - loss: 10.9767 - prescription_output_accuracy: 0.0025 - prescription_output_loss: 6.0029   
Epoch 5/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - disease_output_accuracy: 0.0344 - disease_output_loss: 4.8874 - loss: 10.8492 - prescription_output_accurac

In [21]:
def predictions(pat_problem):
    #preprocess the input
    sequence=tokenizer.texts_to_sequences([pat_problem])
    padded_seqe=pad_sequences(sequence,maxlen=max_length,padding='post')

    #make predictions
    prediction=model.predict(padded_seqe)

    #decode predictions 
    dis_index=np.argmax(prediction[0],axis=1)[0]
    pre_index=np.argmax(prediction[1],axis=1)[0]
    dis_pred=label_encode_disease.inverse_transform([dis_index])[0]
    pre_pred=label_encode_prescription.inverse_transform([pre_index])[0]

    print(f"Predicted Disease:{dis_pred}")
    print(f"Suggested Prescription:{pre_pred}")

input="I ma facing ring worm skin infection"
predictions(input)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Predicted Disease:Eczema
Suggested Prescription:Moisturizers; topical steroids.


In [ ]:
model.save('health_model.h5')



In [ ]:
#load the model 
loaded_model = load_model('health_model.h5')
